# In this script the trained models from SHAPR are beeing applied on the RBC images provided by the Swiss Institute for Translational and Entrepreneurial Medicine (SITEM)

@article{waibel2022shapr, title={SHAPR predicts 3D cell shapes from 2D microscopic images}, author={Waibel, Dominik JE and Kiermeyer, Niklas and Atwell, Scott and Sadafi, Ario and Meier, Matthias and Marr, Carsten}, journal={Iscience}, pages={105298}, year={2022}, publisher={Elsevier} }

https://www.cell.com/iscience/fulltext/S2589-0042(22)01570-X?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS258900422201570X%3Fshowall%3Dtrue

In [ ]:
# !pip3 install shapr 
# uncomment above code if shapr hasn't been installed yet

### Import dependencies

In [ ]:
from shapr.utils import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from shapr import settings
from shapr.data_generator import *
from shapr.model import model, define_adverserial
import os
import tifffile

### Define the paths where the folders "image" and "mask" are

In [ ]:
#change according to your folder-setup
# had to store the results on my private laptop because onedrive which is connected to my github had not enough GB
#settings.path:  str = r"C:\Users\Domin\OneDrive\Dokumente\GitHub\BachelorThesis"
#settings.result_path: str= r"C:\Users\Domin\OneDrive\Dokumente\GitHub\BachelorThesis\Results"
#settings.pretrained_weights_path: str = r"C:\Users\Domin\OneDrive\Dokumente\GitHub\BachelorThesis\logs"

settings.path:  str = r"C:\6.Semester\BTHE\Experiment_Folders\11000_GAN1_images"
settings.result_path: str= r"C:\6.Semester\BTHE\Experiment_Folders\11000_GAN1_images\Results"
settings.pretrained_weights_path: str = r"C:\6.Semester\BTHE\Experiment_Folders\11000_GAN1_images\logs"

In [ ]:
test_filenames =  os.listdir(os.path.join(settings.path, "mask"))
print(test_filenames)
print(len(test_filenames))

### Load pretrained_weights


In [ ]:
PARAMS = {"num_filters": 10,
      "dropout": 0.
}

#change the pretrained_weights here, need to change accordingly if the file is called differently
pretrained_weightsPredictor_file = os.path.join(settings.path, "logs", "pretrained_weightsPredictor_GAN"+str(1) +".hdf5")
SHAPRmodel, discriminator = model(PARAMS)
SHAPRmodel.load_weights(pretrained_weightsPredictor_file)

### Create prediction

In [ ]:
test_data = data_generator_test_set(settings.path, test_filenames)

predict = SHAPRmodel.predict(test_data, steps = len(test_filenames))
print("finished")

### Save as tif-files

In [ ]:
for i in range(len(predict)):
    result = predict[i] 

    result = np.squeeze(result)
    tifffile.imwrite("Results/"+str(test_filenames[i][:-3]) +"tif", result)

### Visualizing volumina example

In [ ]:
# https://github.com/marrlab/SHAPR_torch/blob/master/shapr/scripts/tif_to_x3d.py
# in this script the volumina isn't exported as x3d file because trying it generated error for some reasons
from vedo import Plotter
from vedo import Volume
from shapr.utils import import_image
from matplotlib import pyplot as plt1

image_data = import_image("Results/cell_frame004185_x0513_y0217_red.tif").squeeze()

def to_x3d(image_data):
    plt = Plotter(size=(600, 600), bg='white', offscreen=False, interactive=True)
    volume = Volume(image_data).isosurface()

    # Add some nice colours based on the y coordinate. This is just for
    # visualisation purposes.
    coords = volume.points()
    volume.cmap('Spectral', coords[:, 1])

    plt.show(volume, axes=1, viewup= "z", interactive = True)

   

to_x3d(image_data)




With the python script "BTHE_3D_Visualization" it is possible to have a 360° video of a predicted cell in an interactive Window